In [ ]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
config = yaml.safe_load(Path(params["config_path"]).open("r"))
song_path = Path(params["song_path"])
RenderJSON(config)

In [ ]:
ds = xr.Dataset()
if config["processing"]["load"]["method"] == "wav":
    import scipy.io.wavfile
    fs, data = scipy.io.wavfile.read(song_path)
    data = np.squeeze(data)
    if data.ndim > 1:
        raise Exception("Only single channel wav handled")
    ds["song"] = xr.DataArray(data, dims="t")
    ds["t"] = np.arange(data.size)/fs
    ds["t"].attrs["fs"] = fs
else:
    raise Exception(f'Unknown load method {config["processing"]["load"]["method"]}')
display(ds)


In [ ]:
if "filter" in config["processing"]:
    if config["processing"]["filter"]["method"] == "butter":
        import scipy.signal
        filter_params = config["processing"]["filter"]["method_params"]
        filter = scipy.signal.butter(filter_params["order"], [filter_params["min_freq"], filter_params["max_freq"]], "bandpass", fs=fs, output="sos")
        ds["filtered_song"] = xr.DataArray(scipy.signal.sosfiltfilt(filter, ds["song"].to_numpy()), dims="t")
    else:
        raise Exception(f'Unknown load method {config["processing"]["filter"]["method"]}')
else:
    ds["filtered_song"] = ds["song"]
ds

In [ ]:
if "resample" in config["processing"]:
    if config["processing"]["resample"]["method"] == "interpolation":
        filter_params = config["processing"]["resample"]["method_params"]
        new_fs = pd.DataFrame([dict(song_fs=fs)]).eval(filter_params["out_fs_expr"]).iat[0]
        if new_fs != fs:
            ds["resampled_song"] = ds["filtered_song"].interp(dict(t=np.arange(int(ds["t"].max().item()*new_fs))/new_fs)).rename(t="new_t")
            ds["new_t"].attrs["fs"] = new_fs
        else:
            ds["resampled_song"] = ds["filtered_song"].rename(t="new_t")
            ds["new_t"].attrs["fs"] = new_fs
    else:
        raise Exception(f'Unknown resample method {config["processing"]["resample"]["method"]}')
else:
    ds["resampled_song"] = ds["filtered_song"].rename(t="new_t")
    ds["new_t"].attrs["fs"] = fs
ds

In [ ]:
if config["processing"]["dump"]["method"] == "wav":
    import scipy.io.wavfile
    scipy.io.wavfile.write(song_path.name, ds["new_t"].attrs["fs"], ds["resampled_song"].to_numpy().astype(np.int16))
else:
    raise Exception(f'Unknown dump method {config["processing"]["dump"]["method"]}')